In [135]:
import pandas as pd
import numpy as np
import math
import datetime

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60


In [136]:
def new_route(schedule):
    times = []
    keys = []
    row = dft[get_row(dft, 'Cremona, sesto 39').values]
    key1 = ''
    key2 = ''
    impossibile = []
    
    for i in range(1,len(schedule)):
        key1 = row.index[0]
        try:
            paese=schedule[i]
        except:
            continue
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    
    steps_times1 = pd.Series(times , keys)
    times1 = times[:]
    return steps_times1

In [137]:
#first attempt to local search algorithm
def find_best_position(steps_times, to_insert):
    times = []
    key2 = []
    for i in range(len(steps_times)):
        cities = steps_times.keys()[i].split('-')
        row1 = dft[get_row(dft, cities[0].strip()).values]
        #time from city 1 to current (to insert)
        f_c1_to_curr = row1.loc[:,get_column(dft, to_insert).values].values[0][0]
        row2 = dft[get_row(dft, to_insert).values]
        #time from current (to insert) to city 2
        f_curr_to_c2 = row2.loc[:,get_column(dft, cities[1].strip()).values].values[0][0]
        times.append(f_c1_to_curr + f_curr_to_c2)
    s3 = pd.Series(times, steps_times.keys())
    return s3

In [41]:
"""#furgoncino 1 (MON, FRI)
def local_search1(i, day):
    fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")

    schedule = []
    schedule.append('Cremona, Sesto 39'.lower())
    for j in range(len(fur1[day].dropna())):
        schedule.append(dft[get_row(dft, fur1[day][j]).values].index[0].lower())

    schedule.append('Ostiano, Mazzini, 1'.lower())
    #schedule.append('San Giovanni in Croce, Feudatari, 25 A'.lower())

    #print(schedule)
    l = len(schedule)-2
    moved = []
    print(schedule[i])
    #finché in moved non ci sono tutte le città (meno Cremona e Ostiano)
    while len(moved) < l:
        #faccio il pop della città
        to_insert = schedule.pop(i)
        #controllo se l'ho già considerata
        if(to_insert not in moved):
            #(se no) la inserisco nella lista di quelle già considerate
            moved.append(to_insert)
            #calcolo la nuova route tra le città rimanenti
            route = new_route(schedule, day)
            #trovo la posizione migliore
            best = find_best_position(route, to_insert).idxmin()
            city = best.split('-')[0].strip().lower()
            try:
                #trovo l'indice della posizione migliore
                position = schedule.index(city) + 1
            except:
                #problema nome diverso
                if(city == 'casale cremasco'):
                    city = 'casale cremasco-vidolasco, roma, 56'
                    position = schedule.index(city) + 1
            #inserisco la città nella posizione corretta
            schedule.insert(position ,to_insert)
        else:
            #(se si) reinserisco la città nella sua posizione e vado avanti
            schedule.insert(i ,to_insert)
            #incremento l'indice per la prossima città
            i += 1
            i = i%(l+1)
            if(i == 0):
                i = 1
    return schedule"""

'#furgoncino 1 (MON, FRI)\ndef local_search1(i, day):\n    fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")\n\n    schedule = []\n    schedule.append(\'Cremona, Sesto 39\'.lower())\n    for j in range(len(fur1[day].dropna())):\n        schedule.append(dft[get_row(dft, fur1[day][j]).values].index[0].lower())\n\n    schedule.append(\'Ostiano, Mazzini, 1\'.lower())\n    #schedule.append(\'San Giovanni in Croce, Feudatari, 25 A\'.lower())\n\n    #print(schedule)\n    l = len(schedule)-2\n    moved = []\n    print(schedule[i])\n    #finché in moved non ci sono tutte le città (meno Cremona e Ostiano)\n    while len(moved) < l:\n        #faccio il pop della città\n        to_insert = schedule.pop(i)\n        #controllo se l\'ho già considerata\n        if(to_insert not in moved):\n            #(se no) la inserisco nella lista di quelle già considerate\n            moved.append(to_insert)\n            #calcolo la nuova route tra le città rimanenti\n            route

In [138]:
#returns a pd.Series: keys = city1 - city2, values = times from city1 to city2
def schedule_time(new_schedule):
    times = []
    row = dft[get_row(dft, 'Cremona, sesto 39').values]
    keys = []
    key1 = ''
    key2 = ''
    for i in range(1,len(new_schedule)):
        key1 = row.index[0]
        paese=new_schedule[i]

        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    steps_times = pd.Series(times , keys)
    return steps_times

In [43]:
"""fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
day = 'venerdi'
schedule = []
schedule.append('Cremona, Sesto 39'.lower())
for i in range(len(fur1[day].dropna())):
    schedule.append(dft[get_row(dft, fur1[day][i]).values].index[0].lower())
schedule.append('Ostiano, Mazzini, 1'.lower())
    
best_serie = schedule_time(schedule)
best_time = sum(best_serie.values)

for i in range(1, len(schedule)-1):
    new_schedule = local_search1(i, day)
    time = sum(schedule_time(new_schedule).values)
    print(time)
    if(time < best_time):
        best_time = time
        best_serie = new_schedule
        print('scambiato')
        
print(best_time)
print(best_serie)"""

'fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")\nday = \'venerdi\'\nschedule = []\nschedule.append(\'Cremona, Sesto 39\'.lower())\nfor i in range(len(fur1[day].dropna())):\n    schedule.append(dft[get_row(dft, fur1[day][i]).values].index[0].lower())\nschedule.append(\'Ostiano, Mazzini, 1\'.lower())\n    \nbest_serie = schedule_time(schedule)\nbest_time = sum(best_serie.values)\n\nfor i in range(1, len(schedule)-1):\n    new_schedule = local_search1(i, day)\n    time = sum(schedule_time(new_schedule).values)\n    print(time)\n    if(time < best_time):\n        best_time = time\n        best_serie = new_schedule\n        print(\'scambiato\')\n        \nprint(best_time)\nprint(best_serie)'

In [139]:
#furgoncino 1 (MON, FRI)
def local_search2(i, day, last_city, fur):
    #fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
    schedule = []
    schedule.append('Cremona, Sesto 39'.lower())
    for j in range(len(fur[day].dropna())):
        schedule.append(dft[get_row(dft, fur[day][j]).values].index[0].lower())
    schedule.append(last_city.lower())
    #schedule.append('San Giovanni in Croce, Feudatari, 25 A'.lower())

    schedule_pre = schedule[:]
    schedule_pre_time = sum(schedule_time(schedule_pre).values)
    #print(f"first pre time: {schedule_pre_time}")
    #print(schedule)
    l = len(schedule)-2
    moved = []
    #print(schedule[i])
    #finché in moved non ci sono tutte le città (meno Cremona e Ostiano)
    while len(moved) < l:
        #faccio il pop della città
        to_insert = schedule.pop(i)
        #controllo se l'ho già considerata
        if(to_insert not in moved):
            #(se no) la inserisco nella lista di quelle già considerate
            moved.append(to_insert)
            #calcolo la nuova route tra le città rimanenti
            route = new_route(schedule)
            #trovo la posizione migliore
            best = find_best_position(route, to_insert).idxmin()
            city = best.split('-')[0].strip().lower()
            try:
                #trovo l'indice della posizione migliore
                position = schedule.index(city) + 1
            except:
                #problema nome diverso
                if(city == 'casale cremasco'):
                    city = 'casale cremasco-vidolasco, roma, 56'
                    position = schedule.index(city) + 1
                elif(city == 'gadesco'):
                    city = 'gadesco-pieve delmona, giovanni lonati 9'
                    position = schedule.index(city) + 1
                else:
                     print('PROBLEM')
            #inserisco la città nella posizione corretta
            schedule.insert(position ,to_insert)
            new_time = sum(schedule_time(schedule).values)
            #print(new_time)
            if(new_time < schedule_pre_time):
                schedule_pre = schedule[:]
                schedule_pre_time = new_time
                #print('scambiato')
            else:
                schedule = schedule_pre[:]
        else:
            #(se si) reinserisco la città nella sua posizione e vado avanti
            schedule.insert(i ,to_insert)
            #incremento l'indice per la prossima città
            i += 1
            i = i%(l+1)
            if(i == 0):
                i = 1
    return schedule_pre

In [140]:
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")
def compute_ls(fur, last_city, k):
    if(k == 1):
        week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
    if(k == 2):
        week= ['martedi', 'mercoledi', 'giovedi']
    data={}
    df=pd.DataFrame(data)
    for day in week:
        print(day)

        schedule = []
        schedule.append('Cremona, Sesto 39'.lower())
        for i in range(len(fur[day].dropna())):
            schedule.append(dft[get_row(dft, fur[day][i]).values].index[0].lower())
        schedule.append(last_city.lower())

        best_serie = schedule_time(schedule)
        best_schedule = schedule[:]
        best_time = sum(best_serie.values)
        
        greedy_schedule = schedule[:]
        greedy_time = sum(best_serie.values)
        
        for i in range(1, len(schedule)-1):
            new_schedule = local_search2(i, day, last_city, fur)
            #print(new_schedule)
            time = sum(schedule_time(new_schedule).values)
            #print(time)
            if(time < best_time):
                best_schedule = new_schedule[:]
                best_time = time
                best_serie = schedule_time(best_schedule)
                #print('scambiato')

        #print(best_time)
        #print(best_serie)
        #print(best_schedule)
        if(k == 1):
            csv_shifts = 'ls1_shifts.csv'
            df[day] = pd.Series(best_schedule)
        if(k == 2):
            csv_shifts = 'ls2_shifts.csv'
            df[day] = pd.Series(best_schedule)
    df.to_csv('../Dati/turni_f/' + csv_shifts, index=False, sep=";")

In [76]:
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
compute_ls(fur1, 'Ostiano, Mazzini, 1', 1)

lunedi
sesto ed uniti, roma, 4
grumello cremonese ed uniti, roma, 2
soresina, giacomo matteotti, 6
trigolo, roma, 26
castelleone, roma, 67
montodine,  xxv aprile, 10
ripalta cremasca, roma, 5
ricengo, guglielmo marconi
sergnano, iv novembre,  10
casale cremasco-vidolasco, roma, 56
camisano, trieste, 57
capralba, piave, 2
soncino, dante alighieri, 23
pizzighettone, giuseppe garibaldi, 18
martedi
cremona, ugolani dati, 4
persico dosimo, albert ,7
ostiano, mazzini, 1
san giovanni in croce, feudatari, 25 a
scandolara ravara, italia
casalmaggiore, guglielmo marconi, 8
casalpusterlengo, pusterla 7
crema, civerchi, 9
vailate, dante alighieri, 7
mercoledi
castelverde, ubaldo ferrari, 15
annicco, giuseppe garibaldi, 6
castelleone, roma, 67
madignano, portici, 4
bagnolo cremasco, roma, 3
palazzo pignano, marco polo, 31
pandino, castello, 15
rivolta d'adda, stefano renzi, 5
spino d'adda, circonvallazione, 13
romanengo, guaiarini, 14
giovedi
cremona, sesto 39
cremona, ugolani dati, 4
persico dosim

In [141]:
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")
compute_ls(fur2, 'San Giovanni in Croce, Feudatari, 25 A', 2)

martedi
mercoledi
giovedi
